In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

### Алгоритм KNN


In [2]:
def knn_distances(xTrain,xTest,k):
    #Евклидово расстояние
    distances = -2 * xTrain@xTest.T + np.sum(xTest**2,axis=1) + np.sum(xTrain**2,axis=1)[:, np.newaxis]
    #Заменяем отрицательные числа на 0
    distances[distances < 0] = 0
    distances = distances**.5
    indices = np.argsort(distances, 0) #получаем индексы
    distances = np.sort(distances,0) #отсортированные расстояния
    #возвращаем k ближайших соседей
    return indices[0:k,:], distances[0:k,:]


def knn_predictions(xTrain,yTrain,xTest,k=3):
    indices, distances = knn_distances(xTrain,xTest,k)
    yTrain = yTrain.flatten()
    rows, columns = indices.shape
    predictions = list()
    for j in range(columns):
        temp = list()
        for i in range(rows):
            cell = indices[i][j]
            temp.append(yTrain[cell])
        predictions.append(max(temp,key=temp.count))
    predictions=np.array(predictions)
    return predictions

def knn_accuracy(yTest,predictions):
    x=yTest.flatten()==predictions.flatten()
    grade=np.mean(x)
    return np.round(grade*100,2)

### Подготовка данных

In [3]:
def transform_data(data):
    data['Пол'] = data['Пол'].replace(pol)
    data['Кто_ты'] = data['Кто_ты'].replace(who_are_you)
    data['Дом'] = data['Дом'].replace(dom)
    data['Характер'] = data['Характер'].replace(haracter)
    data['Работаете'] = data['Работаете'].replace(work)
    data['Доход'] = data['Доход'].replace(dohod)
   

In [4]:
data=pd.read_excel('data.xlsx')

In [5]:
data

,Возраст,Пол,Кто_ты,Подъем,Дом,Характер,Работаете,Доход,Присутствие
0,22,ж,Сова,8.0,Да,Интроверт,Да,Доволен,Да
1,21,м,Сова,9.0,Нет,Интроверт,Да,Не доволен,Нет
2,22,м,Сова,10.0,Да,Интроверт,Нет,Не доволен,Да
3,22,м,Сова,9.0,Да,Экстраверт,Да,Доволен,Нет
4,21,ж,Сова,11.0,Да,Интроверт,Да,Доволен,Да
5,21,м,Сова,10.0,Нет,Интроверт,Да,Доволен,Нет
6,22,м,Сова,6.0,Да,Экстраверт,Да,Не доволен,Нет
7,22,м,Сова,9.0,Да,Интроверт,Нет,Не доволен,Да
8,22,м,Сова,11.0,Да,Интроверт,Да,Доволен,Нет
9,21,м,Жаворонок,7.5,Да,Экстраверт,Да,Не доволен,Нет


In [6]:
pol={'м':1, "ж":0}
who_are_you={'Сова':0, 'Жаворонок':1}
dom={'Да':0, "Нет":1}
haracter={'Интроверт':1, "Экстраверт":0}
work={'Да':1, "Нет":0}
dohod={'Доволен':1, "Не доволен":0}
poseshenie={'Да':1, "Нет":0}

In [7]:
transform_data(data)
data['Присутствие'] = data['Присутствие'].replace(poseshenie)

In [8]:
data

,Возраст,Пол,Кто_ты,Подъем,Дом,Характер,Работаете,Доход,Присутствие
0,22,0,0,8.0,0,1,1,1,1
1,21,1,0,9.0,1,1,1,0,0
2,22,1,0,10.0,0,1,0,0,1
3,22,1,0,9.0,0,0,1,1,0
4,21,0,0,11.0,0,1,1,1,1
5,21,1,0,10.0,1,1,1,1,0
6,22,1,0,6.0,0,0,1,0,0
7,22,1,0,9.0,0,1,0,0,1
8,22,1,0,11.0,0,1,1,1,0
9,21,1,1,7.5,0,0,1,0,0


In [12]:
xTrain, xTest = train_test_split(data, test_size=0.25)
yTrain = np.array(xTrain.iloc[:,7])
xTrain = np.array(xTrain.iloc[:,0:7])
yTest = np.array(xTest.iloc[:,7])
xTest = np.array(xTest.iloc[:,0:7])
xTest

array([[21.,  1.,  0.,  9.,  1.,  1.,  1.],
       [22.,  1.,  0.,  6.,  0.,  0.,  1.],
       [21.,  0.,  0., 11.,  0.,  1.,  1.]])

### Обучение


In [13]:
predictions = knn_predictions(xTrain, yTrain, xTest,2)
predictions

array([1, 0, 1], dtype=int64)

In [14]:
print('Accuracy:',knn_accuracy(predictions,yTest),'%')

Accuracy: 66.67 %


### Предсказание

In [15]:
l=[]
for i in list(data.drop(columns='Присутствие')):
    n=input(f'Ответьте: {i} ')
    l.append(n)

Ответьте: Возраст 22
Ответьте: Пол м
Ответьте: Кто_ты Сова
Ответьте: Подъем 11
Ответьте: Дом Да
Ответьте: Характер Интроверт
Ответьте: Работаете Нет
Ответьте: Доход Доволен


In [16]:
data_test=pd.DataFrame(data=[l], columns=list(data.drop(columns='Присутствие')))
transform_data(data_test)
data_test = data_test.astype({'Возраст': np.int16})
data_test = data_test.astype({'Подъем': np.int16})


In [17]:
xpred = np.array(data_test.iloc[:,0:7])
xpred

array([[22,  1,  0, 11,  0,  1,  0]], dtype=int64)

In [18]:
prediction = knn_predictions(xTrain, yTrain, xpred,3)
print('Не пришел' if prediction[0]==0 else 'Пришел')


Не пришел


### Подбор параметра

In [19]:
from sklearn.metrics import accuracy_score
from sklearn import metrics
import matplotlib.pyplot as plt 

In [20]:
Ks = 15
mean_acc = np.zeros((Ks-1))
std_acc = np.zeros((Ks-1))
for n in range(1,Ks):
    yhat = knn_predictions(xTrain,yTrain,xTest,n)
    mean_acc[n-1] = metrics.accuracy_score(yTest, yhat)
    std_acc[n-1] = np.std(yhat==yTest)/np.sqrt(yhat.shape[0])

print( "Лучший результат:", np.round(mean_acc.max()*100,2),"%, k=", mean_acc.argmax()+1) 

Лучший результат: 66.67 %, k= 1
